<a href="https://colab.research.google.com/github/sanja7s/LLM_uses_AI_technology/blob/main/Comprehensiveness_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
#### Load the API key and relevant Python libaries.

In [ ]:
!pip install langchain
!pip install faiss-cpu
!pip install openai
!pip install unstructured
!pip install python-dotenv
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import io
from dotenv import dotenv_values, load_dotenv, find_dotenv
import openai
import os
import ast
from copy import deepcopy
from itertools import islice
import time
import json
import pprint
import numpy as np

uploaded = files.upload()

Saving env to env


In [ ]:
# Get the first key from the uploaded dictionary
env_file_key = list(uploaded.keys())[0]

# Read the uploaded file
env_content = uploaded[env_file_key].decode('utf-8')

# Load the content into a variable
env_variables = dotenv_values(stream=io.StringIO(env_content))

api_key = env_variables['OPENAI_API_KEY']
openai.api_key = api_key

# Models

In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-4",
                                 temperature=0,
                                 max_tokens=4500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut
    )
    return response.choices[0].message["content"]

In [ ]:
def get_completion_and_token_count(messages,
                                 model="gpt-4",
                                 temperature=0,
                                 max_tokens=4500):

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
    'prompt_tokens':response['usage']['prompt_tokens'],
    'completion_tokens':response['usage']['completion_tokens'],
    'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

## Functions

In [ ]:
GT_uses = 75
LLM_uses = 138

In [ ]:
def combine_and_summarise_results(res0, res1, type_of_prompt=''):

  GT_uses = 75
  LLM_uses = 138

  output = {'Type of Prompt': type_of_prompt}

  res = {}

  res['Strict matches'] = {**res0['Strict matches'], **res1['Strict matches']}
  res['Relaxed matches'] = {**res0['Relaxed matches'], **res1['Relaxed matches']}

  strict_matches = [res['Strict matches'][use] for use in res['Strict matches']]
  flattened_list = [item for sublist in strict_matches for item in sublist]
  strict_matches = set(flattened_list)
  output['Strict matches'] = (len(strict_matches), (len(strict_matches)/GT_uses)*100.0)


  relaxed_matches = [res['Relaxed matches'][use] for use in res['Relaxed matches']]
  flattened_list = [item for sublist in relaxed_matches for item in sublist]
  relaxed_matches = set(flattened_list)
  relaxed_matches = relaxed_matches.union(strict_matches)
  output['Relaxed matches'] = (len(relaxed_matches), (len(relaxed_matches)/GT_uses)*100.0)

  GT =set([x+1 for x in range(GT_uses)])
  strict_GT_misses = GT.difference(strict_matches)
  all_GT_misses = GT.difference(relaxed_matches)
  GT_misses = {'strict GT misses':strict_GT_misses, 'relaxed GT misses' : all_GT_misses}

  misses = len(all_GT_misses)
  output['Misses'] = (misses, (misses/GT_uses)*100.0)

  return output, res, GT_misses


In [ ]:
def read_prompt_output():
  print("Select the right input you need.")
  selected_prompt_uploaded = files.upload() # change this for other prompts

  # Get the first key from the uploaded dictionary
  file_name = list(selected_prompt_uploaded.keys())[0]

  # Read the uploaded file
  file_content = selected_prompt_uploaded[file_name].decode('utf-8')

  file_content_dict = ast.literal_eval(file_content)

  N = len(file_content_dict[0])

  # rename the uses so that we have 46 ids
  i = 0
  for el in file_content_dict:
    for use_el in el:
      use_el['Use'] = int(use_el['Use']) + i * N
    i += 1

  flattened_list = [item for sublist in file_content_dict for item in sublist]
  return flattened_list, file_name

In [ ]:
def perform_single_evaluation(prompt_output, input_name, iterration, chunks):

  ############################################################################
  RESPONSES = []
  chunks = 6
  split_points = [len(prompt_output)*i//chunks for i in range(chunks)] + [len(prompt_output)]
  split_tuples = [(split_points[i],split_points[i+1]) for i in range(chunks)]

  cost = 0

  # split your data
  for (i,j) in split_tuples:

      print ("Comparison for uses {} to {}".format(i,j))
      selected_prompt_subset = prompt_output[i:j]

      messages = deepcopy(MESSAGES)
      messages[1]['content'] += str(selected_prompt_subset)

      # response  = get_completion_from_messages(messages, temperature=0)

      response, token_count = get_completion_and_token_count(messages)
      # print(token_count)
      res = token_count
      cost_chunk = (res['prompt_tokens'] * 0.03  + res['completion_tokens'] * 0.06)/1000.0
      cost += cost_chunk
      # print(response)
      time.sleep(20)

      RESPONSES.append(response)


  print('PROMPT QUERY COMPLETED.')
  tmp0 = ast.literal_eval(RESPONSES[0])

  for i in range(1,len(RESPONSES)):
    print(f"Combining {i} and {len(RESPONSES)}.")
    tmp1 = ast.literal_eval(RESPONSES[i])

    res0 = deepcopy(tmp0)
    res1 = deepcopy(tmp1)

    summary, tmp0, GT_misses = combine_and_summarise_results(res0,res1, type_of_prompt='FRT_2RU')


  full_summary = summary
  res = tmp0

  result_file_name = f"{input_name}_res_{iterration}.json"
  with open(result_file_name, 'w') as json_file:
    json.dump(res, json_file, indent=4)  # 4 spaces of indentation
  # Download the file to your local machine
  files.download(result_file_name)

  summary_file_name = f"{input_name}_summary_{iterration}.json"
  with open(summary_file_name, 'w') as json_file:
      json.dump(full_summary, json_file, indent=4)  # 4 spaces of indentation
  # Download the file to your local machine
  files.download(summary_file_name)

  return cost


## Comparison Prompt

In [ ]:
MESSAGES = [
    {
        'role': 'system',
        'content': """
        You are a professional Data Analyst dedicated to assessing and comparing datasets to ensure that the information they contain is coherent, accurate, and aligned.
        """
    },
    {
        'role': 'user',
        'content': """

        Compare the described uses of facial recognition technology in List A given in JSON format with the items listed in List B.
        Your comparisons contain the following elements: strict matches and relaxed matches.

        Take the following definitions into account and follow the rules described below when comparing the uses described in List A with the items in List B:
        1. Strict match: exact correspondence between a described use of facial recognition technology in List A and an item in List B without any deviations in their similarity.
        For a strict match, the described use of facial recognition technology in List A and the item in List B are indeed the same. Therefore, be strict and conservative in your choice.
        If no strict match was found for a use output an empty list for the specific use.
        2. Relaxed match: partial correspondence between a described use of facial recognition technology in List A and an item in List B.
        While there are significant similarities between the use of List A and an item in List B, they are not a strict match. Be strict and conservative in your choice.
        Uses already identified as strict matches cannot be listed as relaxed matches for the same use from List A. If no relaxed match was found for a use output an empty list for the specific use.

        Output the following JSON:
        {"Strict matches":
          { "Use 1": [list the item numbers in List B that are strict matches for Use 1 from List A],
           "Use 2": [list the item numbers in List B that are strict matches for Use 2 from List A],
          continue the list...
          },
        "Relaxed matches":
          { "Use 1": [list the item numbers in List B that are relaxed matches for Use 1 from List A],
          "Use 2": [list the item numbers in List B that are relaxed matches for Use 2 from List A],
          continue the list...
          }
        }
        MAKE SURE TO OUTPUT *ONLY* the JSON file and nothing else.

        List B:
        1. Access control for buildings, areas, storage places, banks' vaults and lockers.
        2. Access control for technology, secure networks, and resources.
        3. Detect unauthorized personnel.
        4. Securely register and manage visitors.
        5. Provide real-time information about visitors in high-profile buildings.
        6. Identify men in female-reserved coaches or women-only areas.
        7. Visualize building occupancy and peak hours and determine foot traffic patterns.
        8. Identify individuals approaching homes for example through smart doorbells.
        9. Trigger an alarm when unidentified individuals enter a home.
        10. Enable unlocking devices and apps.
        11. Enable unlocking cars and driving them.
        12. Notify owners if someone attempts to steal their car.
        13. Track worker and student attendance.
        14. Check if students and workers comply with regulations.
        15. Monitor and track students' activities and performance to aid university examinations.
        16. Identify demographic attributes of individuals, including gender, age, ethnicity, and sexual orientation.
        17. Perform profiling to identify patterns and characteristics of individuals or groups.
        18. Implement targeted recognition systems that customize responses or services based on a person's ethnicity or gender.
        19. Customize advertisements and promotions for targeted marketing.
        20. Identify individuals' shopping habits for personalized discounts and offerings.
        21. Evaluate consumer satisfaction.
        22. Monitor wait times and moods in check-out or customer service lines.
        23. Personalized recommendations, surroundings, and services for smart homes, automotive environments, and travel industry.
        24. Identify and track criminals, suspects, stalkers, or terrorists.
        25. Recording an arrested individual's facial information.
        26. Gathering, analyzing, and interpreting evidence from a crime scene or incident.
        27. Register traffic violations by rental transport users.
        28. Establish a unified penalty system, collecting fines for various violations such as fare dodging.
        29. Define groups of people and create whitelists for VIPs and blacklists for unwanted individuals.
        30. Compare individuals against watchlists containing names, identifiers, or attributes of known individuals of interest or potential risks.
        31. Prevent child exploitation and abduction.
        32. Search for and identify missing persons.
        33. Identify abusive law enforcement officers.
        34. Implement face tagging in images.
        35. Create digital photo books.
        36. Alert individuals when photographs with their faces are posted online.
        37. Detect and report inappropriate pictures using facial recognition and pattern analysis.
        38. Group photos based on individuals present.
        39. Conduct face scans to search for specific individuals in pictures.
        40. Use selfies to find users' doppelganger in a database of recognized paintings.
        41. Recognize when fake profiles use someone else's face.
        42. Offer friend suggestions on social media platforms.
        43. Prevent online dating fraud.
        44. Enable face authentication-based mobile payments and other banking services.
        45. Compare a customer's face during ATM usage with database to reduce fraudulent activities.
        46. Payments on public transport (E.g. metro trains, public buses, toll gates).
        47. Identify patients and facilitate check-in and other processes for patients, for example, provide notifications to respective doctors, generate e-prescriptions for patients, update recent improvements in a patient's case study.
        48. Identify healthcare staff.
        49. Track staff and patients to keep a record of the movement and presence of both staff members and patients within a healthcare facility.
        50. Streamline and manage medicine distribution in healthcare settings.
        51. Diagnose or support detection of diseases in individuals.
        52. Retrieve critical information of people in emergencies, such as their blood group.
        53. Conduct real-time mental health tests.
        54. Provide automatic floor selection in elevators for elderly and individuals with disabilities.
        55. Assist individuals with impairments by identifying their friends and people, providing reminders of names, and relevant information about them.
        56. Assist individuals with visual impairments in taking photos.
        57. Initiate robot operations only when it recognizes an operator in its workspace.
        58. Enable assistive robots to recognize individuals in a home environment.
        59. Facilitate voter identification processes.
        60. Provide tailored learning experiences to suit individual needs.
        61. Create multimedia content using facial recognition.
        62. Identify and differentiate between various characters and actors in movies, making it highly beneficial for content discovery and delivery platforms that seek to provide content based on specific characters or actors.
        63. Overlay cosmetic changes for users.
        64. Use photo and video filters and special beautification effects.
        65. Streamline check-ins and boarding and reduce waiting times at airports.
        66. Identity travelers at border crossings to automate border crossing procedures (traveler identification, biometric passport, passport-checking).
        67. Verify documents, such as passports, visas, and driver's licenses.
        68. Identify and verify people in train stations and stadiums.
        69. Detect fatigue in individuals.
        70. Detect emotions in individuals.
        71. Facilitate recruitment processes by informing hiring decisions and help job interviewers to view candidate’s previous records.
        72. Help people recognize faces by using smart glasses to display names and social network activities of identified people.
        73. Monitor and surveil people.
        74. Facilitate tourists in meeting new people.
        75. Liveness checks that helps prevent hackers from impersonating a client.

        List A:

        """
    }
]

## Perform Comparison

### Final_Variations

In [ ]:
"""
  COST EVALUATION
"""
prompt_output, file_name = read_prompt_output()

# comparisson_cost = perform_single_evaluation(prompt_output, file_name, iterration=1, chunks=4)

# comparisson_cost

Select the right input you need.


Saving Var_2_2RU_T0 (4).json to Var_2_2RU_T0 (4).json


In [ ]:
start_time = time.time()
chunks = 4
for iterration in range(1):
  perform_single_evaluation(prompt_output, file_name, iterration, chunks)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"The command took {elapsed_time} seconds to execute.")

Comparison for uses 0 to 15
Comparison for uses 15 to 30
Comparison for uses 30 to 46
Comparison for uses 46 to 61
Comparison for uses 61 to 76
Comparison for uses 76 to 92
PROMPT QUERY COMPLETED.
Combining 1 and 6.
Combining 2 and 6.
Combining 3 and 6.
Combining 4 and 6.
Combining 5 and 6.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The command took 294.08997869491577 seconds to execute.


## Calculate Extra Uses

In [ ]:
def read_comparison_res():
  print("Select the right input you need.")
  selected_res = files.upload() # change this for other prompts

  # Get the first key from the uploaded dictionary
  file_key = list(selected_res.keys())[0]

  # Read the uploaded file
  file_content = selected_res[file_key].decode('utf-8')

  file_content_dict = ast.literal_eval(file_content)

  return file_content_dict

In [ ]:
def parse_extras(json_res):

  count_extras = 0
  extra_use_ids = []

  res = json_res['Relaxed matches']
  total = len(res)
  for k in res.keys():
    if len(res[k]) == 0:
      count_extras += 1
      extra_use_ids.append(k)

  return count_extras, (count_extras/total)*100.00, extra_use_ids

In [ ]:
results1 = read_comparison_res()

Select the right input you need.


Saving FRT_Final_Variation_1_3RU_T0_res_1 (1).json to FRT_Final_Variation_1_3RU_T0_res_1 (1).json


In [ ]:
results2 = read_comparison_res()

Select the right input you need.


Saving FRT_Final_Variation_1_3RU_T0_res_2 (1).json to FRT_Final_Variation_1_3RU_T0_res_2 (1).json


In [ ]:
results3 = read_comparison_res()

Select the right input you need.


Saving FRT_Final_Variation_1_3RU_T0_res_3 (1).json to FRT_Final_Variation_1_3RU_T0_res_3 (1).json


In [ ]:
results4 = read_comparison_res()

Select the right input you need.


Saving FRT_Final_Variation_1_3RU_T0_res_4 (1).json to FRT_Final_Variation_1_3RU_T0_res_4 (1).json


In [ ]:
results5 = read_comparison_res()

Select the right input you need.


Saving FRT_Final_Variation_1_3RU_T0_res_5 (1).json to FRT_Final_Variation_1_3RU_T0_res_5 (1).json


In [ ]:
total_extras, percent_extras = [], []
for results in [results1, results2, results3, results4, results5]:
  extras, pct_extras, extra_use_ids = parse_extras(results)
  total_extras.append(extras)
  percent_extras.append(pct_extras)

In [ ]:
extras, pct_extras, extra_use_ids = parse_extras(results1)

In [ ]:
extras, pct_extras, extra_use_ids

(53,
 38.405797101449274,
 ['Use 9',
  'Use 10',
  'Use 18',
  'Use 23',
  'Use 25',
  'Use 33',
  'Use 35',
  'Use 36',
  'Use 37',
  'Use 38',
  'Use 40',
  'Use 41',
  'Use 42',
  'Use 44',
  'Use 45',
  'Use 48',
  'Use 49',
  'Use 50',
  'Use 51',
  'Use 55',
  'Use 57',
  'Use 58',
  'Use 59',
  'Use 63',
  'Use 66',
  'Use 69',
  'Use 70',
  'Use 72',
  'Use 74',
  'Use 75',
  'Use 84',
  'Use 86',
  'Use 90',
  'Use 94',
  'Use 95',
  'Use 96',
  'Use 97',
  'Use 98',
  'Use 99',
  'Use 100',
  'Use 104',
  'Use 105',
  'Use 108',
  'Use 112',
  'Use 114',
  'Use 118',
  'Use 119',
  'Use 120',
  'Use 123',
  'Use 124',
  'Use 128',
  'Use 129',
  'Use 135'])

In [ ]:
np.array(total_extras).mean(), np.array(total_extras).std()

(55.2, 4.707440918375928)

In [ ]:
np.array(percent_extras).mean(), np.array(percent_extras).std()

(40.0, 3.411189071286904)